<a href="https://colab.research.google.com/github/Zr695/IMLO-OpenAssessment/blob/main/IMLO_OpenAssessment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torchvision
# %pip install -U git+https://github.com/szagoruyko/pytorchviz.git@master
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor, ToPILImage
from torch.autograd import Variable
from google.colab import runtime
# from torchviz import make_dot

data_transform = transforms.Compose([
    transforms.Resize((500, 500)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=20),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    ])

training_data = datasets.Flowers102(
    root="data",
    split='train',
    download=True,
    transform=data_transform
)
validation_data = datasets.Flowers102(
    root="data",
    split='val',
    download=True,
    transform=transforms.Compose([transforms.Resize((500, 500)), transforms.ToTensor()])
)
test_data = datasets.Flowers102(
    root="data",
    split='test',
    download=True,
    transform=transforms.Compose([transforms.Resize((500, 500)), transforms.ToTensor()])
)


train_dataloader = DataLoader(training_data, batch_size=128)
validate_dataloader = DataLoader(validation_data, batch_size=128)
test_dataloader = DataLoader(test_data, batch_size=128)


class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout(0.05),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            # nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1),
            # nn.ReLU(),
            # nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Flatten(),
            nn.Linear(64 * 125 * 125, 256),
            nn.ReLU(),
            nn.Linear(256, 102)

        )
    def __len__(self):
        return len(self.data)

    def forward(self, x):
        logits = self.layers(x)
        return logits

model = NeuralNetwork()
dev = torch.device(
    "cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f"Using device:{dev}")


100%|██████████| 344862509/344862509 [01:42<00:00, 3380961.43it/s]


Extracting data/flowers-102/102flowers.tgz to data/flowers-102


100%|██████████| 502/502 [00:00<00:00, 568450.49it/s]


100%|██████████| 14989/14989 [00:00<00:00, 20143679.16it/s]


Using device:cuda


In [3]:
# dummy_input = torch.randn(128, 3, 500, 500)
# make_dot(
#     model(dummy_input),
#     params=dict(model.named_parameters())).render("nnSVG", format="pdf"
#     )

In [4]:
learning_rate = 0.0007
batch_size = 128
epochs = 100

In [5]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [6]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 2 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
  model.eval()
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  test_loss, correct = 0, 0

  with torch.no_grad():
      for X, y in dataloader:
          pred = model(X)
          test_loss += loss_fn(pred, y).item()
          correct += (pred.argmax(1) == y).type(torch.float).sum().item()

  test_loss /= num_batches
  correct /= size
  print(f"Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(validate_dataloader, model, loss_fn)
print(f"Final Test Accuracy:")
test_accuracy = test_loop(test_dataloader, model, loss_fn)
torch.save(model.state_dict(),'flowers-102.pth')
runtime.unassign()


Epoch 1
-------------------------------
loss: 4.631837  [  128/ 1020]
loss: 4.752569  [  384/ 1020]
loss: 4.897707  [  640/ 1020]
loss: 4.755946  [  896/ 1020]
Accuracy: 1.4%, Avg loss: 4.624297 

Epoch 2
-------------------------------
loss: 4.622291  [  128/ 1020]
loss: 4.678061  [  384/ 1020]
loss: 4.630188  [  640/ 1020]
loss: 4.661627  [  896/ 1020]
Accuracy: 1.3%, Avg loss: 4.623082 

Epoch 3
-------------------------------
loss: 4.621310  [  128/ 1020]
loss: 4.654305  [  384/ 1020]
loss: 4.599571  [  640/ 1020]
loss: 4.657775  [  896/ 1020]
Accuracy: 1.2%, Avg loss: 4.620535 

Epoch 4
-------------------------------
loss: 4.602549  [  128/ 1020]
loss: 4.669586  [  384/ 1020]
loss: 4.589456  [  640/ 1020]
loss: 4.648163  [  896/ 1020]
Accuracy: 2.5%, Avg loss: 4.615544 

Epoch 5
-------------------------------
loss: 4.594187  [  128/ 1020]
loss: 4.684540  [  384/ 1020]
loss: 4.585526  [  640/ 1020]
loss: 4.638796  [  896/ 1020]
Accuracy: 3.0%, Avg loss: 4.608410 

Epoch 6
-------